In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
q = """
WITH formstack_data AS (
    SELECT
        lower(email_address) AS email,
        form_name,
        min(date(submission_timestamp)) AS formstack_date
    FROM
        custom.formstack_v_community_events_all
    GROUP BY
        email,
        form_name),
ticket_data AS (
    SELECT
        lower(email) as email,
        min(date(transaction_datetime)) AS min_ticket_date
    FROM
        custom.cth_v_historical_ticket ticket
    JOIN
        custom.korepss_externalsystemtocontact ext ON ticket.purchaser_ticketing_id = ext.externalcontactid
    JOIN
        custom.korepss_contacts contacts ON ext.crmcontactid = contacts.contactid
    GROUP BY
        email
)
SELECT
    *
FROM
    formstack_data
LEFT JOIN
    ticket_data USING (email)
"""

df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [4]:
df

,email,form_name,formstack_date,min_ticket_date
0,johnjayperezfl@aol.com,Riptide Beach Festival,2023-12-03,2022-02-12
1,mtrescastro92@gmail.com,Rockin Rib Run,2021-11-21,2021-04-06
2,latoyasmith.med@gmail.com,Autism Speaks Broward,2023-10-01,2022-03-03
3,diannelamas@yahoo.com,Train + REO Speedwagon,2024-08-21,2021-11-26
4,diannelamas@yahoo.com,ADA Step Out Walk,2023-09-23,2021-11-26
...,...,...,...,...
99093,jprosenbaum@comcast.net,Audacy Music Festival,2021-12-05,None
99094,aalison.fu@gmail.com,Wynwood Marketplace,2022-04-30,None
99095,carleybedillion@gmail.com,Tri Rail Fun Day,2022-02-24,None
99096,carolinesepstein@gmail.com,Winterfest Family Fun Day,2021-11-21,None


In [ ]:
df['min_ticket_date'] = df['min_ticket_date'].fillna(0)

df['pre_formstack_email'] = df.apply(lambda row: 0 if row['min_ticket_date'] == 0 else (1 if row['formstack_date'] < row['min_ticket_date'] else 0), axis = 1)
df['post_formstack_email'] = df.apply(lambda row: 0 if row['min_ticket_date'] == 0 else (1 if row['formstack_date'] >= row['min_ticket_date'] else 0), axis = 1)
df['no_tickets'] = df.apply(lambda row: 1 if row['min_ticket_date'] == 0 else 0, axis = 1)

In [8]:
df

,email,form_name,formstack_date,min_ticket_date,pre_formstack_email,post_formstack_email,no_tickets
0,johnjayperezfl@aol.com,Riptide Beach Festival,2023-12-03,2022-02-12,1,0,0
1,mtrescastro92@gmail.com,Rockin Rib Run,2021-11-21,2021-04-06,1,0,0
2,latoyasmith.med@gmail.com,Autism Speaks Broward,2023-10-01,2022-03-03,1,0,0
3,diannelamas@yahoo.com,Train + REO Speedwagon,2024-08-21,2021-11-26,1,0,0
4,diannelamas@yahoo.com,ADA Step Out Walk,2023-09-23,2021-11-26,1,0,0
...,...,...,...,...,...,...,...
99093,jprosenbaum@comcast.net,Audacy Music Festival,2021-12-05,0,0,0,1
99094,aalison.fu@gmail.com,Wynwood Marketplace,2022-04-30,0,0,0,1
99095,carleybedillion@gmail.com,Tri Rail Fun Day,2022-02-24,0,0,0,1
99096,carolinesepstein@gmail.com,Winterfest Family Fun Day,2021-11-21,0,0,0,1
